<a href="https://colab.research.google.com/github/miacatharina/mimic-code/blob/main/Final_estimation_estimator_selection_potassium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install econml[dowhy]

### Setup

In [ ]:
import pandas as pd
import os, warnings, random
import warnings
import econml 
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from econml.dml import DML, LinearDML, SparseLinearDML, CausalForestDML
import numpy as np
from itertools import product
from sklearn.linear_model import (Lasso, LassoCV, LogisticRegression,
                                  LogisticRegressionCV,LinearRegression,
                                  MultiTaskElasticNet,MultiTaskElasticNetCV)
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from econml.score import RScorer
from econml.metalearners import TLearner, SLearner, XLearner, DomainAdaptationLearner
from econml.dml import CausalForestDML, LinearDML, NonParamDML
from econml.dr import DRLearner, ForestDRLearner, LinearDRLearner
from joblib import Parallel, delayed
from econml.metalearners import TLearner, SLearner, XLearner
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.base import clone
import logging
import sys
from econml.dml import DML, LinearDML,SparseLinearDML,CausalForestDML, NonParamDML, KernelDML
from matplotlib import pyplot as plt
import copy
from econml.sklearn_extensions.model_selection import GridSearchCVList
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import dowhy
from dowhy import CausalModel
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from econml.sklearn_extensions.linear_model import WeightedLassoCV, WeightedLasso
from econml.cate_interpreter import SingleTreeCateInterpreter
import shap
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import tensorflow as tf
import timeit
from sklearn.model_selection import train_test_split
from econml.cate_interpreter import SingleTreePolicyInterpreter


def seed_everything(seed=123):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    tf.random.set_seed(seed)

seed = 123
seed_everything(seed)

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
#Check the version of TensorFlow you are using
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.652518975999982
GPU (s):
0.09855003499998816
GPU speedup over CPU: 26x
Found GPU at: /device:GPU:0
2.12.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


### Import the data

In [ ]:
XTY = pd.read_csv("/content/XTY.csv")
X = pd.read_csv("/content/X.csv")
Y = pd.read_csv("/content/Y.csv")
T = pd.read_csv("/content/T.csv")
XTY_train = pd.read_csv("/content/XTY_train.csv")
XTY_test = pd.read_csv("/content/XTY_test.csv")
X_train = pd.read_csv("/content/X_train.csv")
X_test = pd.read_csv("/content/X_test.csv")
Y_train = pd.read_csv("/content/Y_train.csv")
Y_test = pd.read_csv("/content/Y_test.csv")
T_train = pd.read_csv("/content/T_train.csv")
T_test = pd.read_csv("/content/T_test.csv")

XTY.set_index("stay_id", inplace=True)
X.set_index("stay_id", inplace=True)
Y.set_index("stay_id", inplace=True)
T.set_index("stay_id", inplace=True)
XTY_train.set_index("stay_id", inplace=True)
XTY_test.set_index("stay_id", inplace=True)
X_train.set_index("stay_id", inplace=True)
X_test.set_index("stay_id", inplace=True)
Y_train.set_index("stay_id", inplace=True)
Y_test.set_index("stay_id", inplace=True)
T_train.set_index("stay_id", inplace=True)
T_test.set_index("stay_id", inplace=True)

T_train = T_train.iloc[:, 0].to_numpy()
T_test = T_test.iloc[:, 0].to_numpy()
X1_train = X_train.iloc[:, 3:4]
X1_test = X_test.iloc[:, 3:4]
XT = pd.concat([X, pd.DataFrame(T, columns=['treated'])], axis=1)
XT_train = pd.concat([X_train, pd.DataFrame(T_train, columns=['treated'])], axis=1)
XT_test = pd.concat([X_test, pd.DataFrame(T_test, columns=['treated'])], axis=1)


#(1) Binary outcome: Hypokalemia within 48 hours 
Y1A = Y.iloc[:, 2].to_numpy()
Y1A_train = Y_train.iloc[:, 2].to_numpy()
Y1A_test = Y_test.iloc[:, 2].to_numpy()

#(1)Continuous outcome: Minimum serum potassium within 48 hours 
Y1B = Y.iloc[:, 0]#.to_numpy()
Y1B_train = Y_train.iloc[:, 0].to_numpy()
Y1B_test = Y_test.iloc[:, 0].to_numpy()

In [ ]:
print(X_train.shape)

(4355, 83)


# Set up Grid Search Baseline

In [ ]:
class GridSearchCVList:

    def __init__(self, estimator_list, param_grid_list, scoring=None,
                 n_jobs=None, iid='warn', refit=True, cv='warn', verbose=0, pre_dispatch='2*n_jobs',
                 error_score='raise', return_train_score=False):
        self._gcv_list = [GridSearchCV(estimator, param_grid, scoring=scoring,
                                       n_jobs=n_jobs, refit=refit, cv=cv, verbose=verbose,
                                       pre_dispatch=pre_dispatch, error_score=error_score,
                                       return_train_score=return_train_score)
                          for estimator, param_grid in zip(estimator_list, param_grid_list)]
        return

    def fit(self, X, y, sample_weight = None, **fit_params):
        self.best_ind_ = np.argmax([gcv.fit(X, y, sample_weight = sample_weight, **fit_params).best_score_ for gcv in self._gcv_list])
        self.best_estimator_ = self._gcv_list[self.best_ind_].best_estimator_
        self.best_score_ = self._gcv_list[self.best_ind_].best_score_
        self.best_params_ = self._gcv_list[self.best_ind_].best_params_
        return self

    def predict(self, X):
        return self.best_estimator_.predict(X)

    def predict_proba(self, X):
        return self.best_estimator_.predict_proba(X)
  
#Cross-validated nuisance and final stage models
def first_stage_prop():
    return GridSearchCVList([LogisticRegression(random_state=123),
                             RandomForestClassifier(random_state=123),
                             GradientBoostingClassifier(random_state=123)],
                             param_grid_list=[{'C': [0.01, .1, 1, 10, 100]},
                                              {'max_depth': [3, 5],
                                               'min_samples_leaf': [10, 50]},
                                              {'n_estimators': [50, 100],
                                               'min_samples_leaf': [10, 30]}],
                             cv=5,
                             scoring='neg_mean_squared_error')
def first_stage_clf():
    return GridSearchCVList([RandomForestClassifier(random_state=123),
                             GradientBoostingClassifier(random_state=123)],
                             param_grid_list=[{'max_depth': [3, 6],
                                               'min_samples_leaf': [10, 50, 100]},
                                              {'max_depth': [3, 6],
                                               'min_samples_leaf': [10, 50, 100]}],
                             cv=5,
                             scoring='neg_mean_squared_error')

def first_stage_reg():
    return GridSearchCVList([Lasso(random_state=123),
                             RandomForestRegressor(random_state=123),
                             GradientBoostingRegressor(random_state=123)],
                             param_grid_list=[{'alpha': [.001, .01, .1, 1, 10]},
                                               {'max_depth': [3, 6],
                                               'min_samples_leaf': [50, 100]},
                                              {'max_depth': [3, 6],
                                               'min_samples_leaf': [50, 100]}],
                             cv=5,
                             scoring='neg_mean_squared_error')

def final_stage():
    return GridSearchCVList([WeightedLasso(random_state=123),
                             RandomForestRegressor(random_state=123)],
                             param_grid_list=[{'alpha': [.001, .01, .1, 1, 10]},
                                              {'max_depth': [3, 6],
                                               'min_samples_leaf': [50, 100]}],
                             cv=5,
                             scoring='neg_mean_squared_error')

## Best nuisance models: Binary outcome 


In [ ]:
model_y = clone(first_stage_clf().fit(X, Y1A).best_estimator_)
print(model_y)

GradientBoostingClassifier(min_samples_leaf=100, random_state=123)


In [ ]:
model_t = clone(first_stage_prop().fit(X, T).best_estimator_)
print(model_t)

GradientBoostingClassifier(min_samples_leaf=10, random_state=123)


In [ ]:
model_yt = clone(first_stage_clf().fit(XT, Y1A).best_estimator_)
print(model_yt)

GradientBoostingClassifier(min_samples_leaf=100, random_state=123)


In [ ]:
#Binary outcomes: Best estimators for propensity model and outcome model
model_ya = GradientBoostingClassifier(min_samples_leaf=100, random_state=123)
model_ta = GradientBoostingClassifier(min_samples_leaf=10, random_state=123)
model_yta = GradientBoostingClassifier(min_samples_leaf=100, random_state=123)

## Best nuisance models: Continuous outcome 

In [ ]:
model_yb = clone(first_stage_reg().fit(X, Y1B).best_estimator_)
print(model_yb)

GradientBoostingRegressor(min_samples_leaf=100, random_state=123)


In [ ]:
model_ytb = clone(first_stage_reg().fit(XT, Y1B).best_estimator_)
print(model_ytb)

GradientBoostingRegressor(min_samples_leaf=100, random_state=123)


In [ ]:
#Continous outcomes: Best estimators for propensity model and outcome model
model_yb = GradientBoostingRegressor(min_samples_leaf=100, random_state=123)
model_tb = GradientBoostingClassifier(min_samples_leaf=10, random_state=123)
model_ytb =GradientBoostingRegressor(min_samples_leaf=100, random_state=123)

# Set up the RScorer

## Fit the RScorer: Binary outcome 

In [ ]:
# Train the RScore on the data 
#Follow the same procedure for selecting the nuisance models for metrics as for CATE estimators 
scorer_a = RScorer(model_y=first_stage_clf(), model_t=first_stage_prop(),
                 discrete_treatment=True, random_state=123)

In [ ]:
scorer_a.fit(Y1A_test, T_test, X=X_test)

## Fit the RScorer: Continuous outcome 

In [ ]:
# Train the RScore on the data 
#Follow the same procedure for selecting the nuisance models for metrics as for CATE estimators 
scorer_b = RScorer(model_y=first_stage_reg(), model_t=first_stage_prop(),
                 discrete_treatment=True, random_state=123)

In [ ]:
scorer_b.fit(Y1B_test, T_test, X=X_test)

# Set up influence function 

In [ ]:
X_train = pd.read_csv("/content/X_train.csv") 
X_test = pd.read_csv("/content/X_test.csv")
Y_train = pd.read_csv("/content/Y_train.csv")
Y_test = pd.read_csv("/content/Y_test.csv")
T_train = pd.read_csv("/content/T_train.csv")
T_test = pd.read_csv("/content/T_test.csv")
XTY_train = pd.read_csv("/content/XTY_train.csv")


XTY_train.set_index("stay_id", inplace=True)
X_train.set_index("stay_id", inplace=True)
X_test.set_index("stay_id", inplace=True)
T_train.set_index("stay_id", inplace=True)
T_test.set_index("stay_id", inplace=True)

#T_train = T_train.iloc[:, 0].to_numpy()
XT_train = pd.concat([X_train, pd.DataFrame(T_train, columns=['treated'])], axis=1)
XT_test = pd.concat([X_test, pd.DataFrame(T_test, columns=['treated'])], axis=1)

In [ ]:
xgb_plugin1 = LogisticRegression(C=10, random_state=123)
xgb_plugin0 = LogisticRegression(C=1, random_state=123)

In [ ]:
x0 = XT_train.loc[XT_train['treated'] == 0]
y0 = XTY_train.loc[XTY_train['treated'] == 0, '0_48_hypokalemia']
xgb_plugin0.fit(x0, y0)

LogisticRegression(C=1, random_state=123)

In [ ]:
x1 = XT_train.loc[XT_train['treated'] != 0]
y1 = XTY_train.loc[XTY_train['treated'] != 0, '0_48_hypokalemia']
xgb_plugin1.fit(x1, y1)

LogisticRegression(C=10, random_state=123)

In [ ]:
y_pred0 = xgb_plugin0.predict(XT_test)
y_pred1 = xgb_plugin1.predict(XT_test)

In [ ]:
#Estimated treatment effect using the plug-in method (difference between the predicted outcomes of two XGBoost models trained separately on the control group and the treatment group)
t_plugin = y_pred1 - y_pred0

In [ ]:
#Compute the propensity score using the gradient boosting classifier model and the treatment variable
gb = GradientBoostingClassifier(min_samples_leaf=10, n_estimators=50,
                           random_state=123)
gb.fit(X_train, T_train)
ps = gb.predict_proba(X_test)[:, 1]
print(ps)

[0.82123387 0.11883833 0.64019843 ... 0.97021595 0.2471682  0.21186484]


In [ ]:
#Use the PS to compute a weight matrix for the treatment effect estimate
T_test = T_test.iloc[:, 0].to_numpy()
a = (T_test - ps)
ident = np.array([1] * len(ps))
c = (ps * (ident - ps))
#weight vector used to adjust the treatment effect estimate based on the probability of receiving the treatment (propensity score)
b = np.array([2] * len(T_test)) * T_test * (T_test - ps) / c

# (A) Estimator selection: Binary outcome


## 1. LinearDML

### 1.1 Select best performing nuisance model class and hyperparameters

In [ ]:
ldml = LinearDML(model_y=model_ya,
                model_t=model_ta,
                random_state=123)

In [ ]:
ldml.fit(Y1A_train, T_train, X=X_train)

In [ ]:
ldml_cate = ldml.effect(X_test)
ldml_cate_inference = ldml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(ldml_cate_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.066       0.015 4.452    0.0         0.042          0.09
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.153          -0.142           0.299
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.154         -0.208          0.375
------------------------------------------


### 1.2 Calculate the robustness

In [ ]:
ldml.fit(Y=Y1A_test, T=T_test, X=X_test)
ldml_cate_test = ldml.effect(X_test)

ERMSE = ((ldml_cate-ldml_cate_test)**2).mean()**(1/2)
print(ERMSE)

0.33025239834659087


### 1.3 Calculate the R-Score 

In [ ]:
rscore = scorer_a.score(ldml)
print(rscore)

0.0036425188041228074


## 2. LinearDRLearner 

### 2.1 Select best performing nuisance model class and hyperparameters

In [ ]:
LinearDR_learner = LinearDRLearner(model_regression=model_yta, model_propensity=model_ta, random_state=123)

In [ ]:
LinearDR_learner.fit(Y1A_train, T_train, X=X_train)

In [ ]:
LinearDR_learner_cate = LinearDR_learner.effect(X_test)
LinearDR_learner_cate_inference = LinearDR_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(LinearDR_learner_cate_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.009       0.014 0.674  0.501        -0.013         0.032
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
     0.12          -0.179             0.2
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.121         -0.246          0.277
------------------------------------------


### 2.2 Calculate the robustness 

In [ ]:
LinearDR_learner.fit(Y=Y1A_test, T=T_test, X=X_test)
LinearDR_learner_cate_test = LinearDR_learner.effect(X_test)

ERMSE = ((LinearDR_learner_cate-LinearDR_learner_cate_test)**2).mean()**(1/2)
print(ERMSE)

0.2963382710580213


### 3.3 Calculate the RScore 

In [ ]:
rscore = scorer_a.score(LinearDR_learner)
print(rscore)

0.032833816869907384


## 3. S-Learner

### 3.1 Select best performing nuisance model class and hyperparameters

In [ ]:
# Instantiate the SLearner
S_learner = SLearner(overall_model= first_stage_clf())

In [ ]:
# Train S_learner
best_estimator_s_learner = S_learner.fit(Y1A_train, T_train, X=X_train).overall_model.best_estimator_
print(best_estimator_s_learner)

GradientBoostingClassifier(min_samples_leaf=100, random_state=123)


In [ ]:
# Estimate treatment effects on test data
S_learner = SLearner(overall_model=best_estimator_s_learner)
S_learner.fit(Y1A_train, T_train, X=X_train, inference='bootstrap')
S_te = S_learner.effect(X_test)

In [ ]:
S_te_inference = S_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(S_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.001       0.031 0.029  0.977        -0.051         0.053
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
     0.03             0.0             0.0
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.044            nan            nan
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 3.2 Calculate the robustness

In [ ]:
S_learner.fit(Y=Y1A_test, T=T_test, X=X_test)
S_te_test = S_learner.effect(X_test)

ERMSE = ((S_te-S_te_test)**2).mean()**(1/2)
print(ERMSE)

0.030303030303030304


### 3.3 Calculate the R-Score 

In [ ]:
rscore = scorer_a.score(S_learner)
print(rscore)

-0.0010123335155265423


## 4. T-learner

### 4.1 Select best performing nuisance model class and hyperparameters

In [ ]:
# Instantiate the T_Learner
T_learner = TLearner(models=first_stage_clf())

In [ ]:
# Train T_learner
T_learner.fit(Y1A_train, T_train, X=X_train)

In [ ]:
# Assuming that T_learner.models is a list of the fitted GridSearchCVList objects
best_estimators = [model.best_estimator_ for model in T_learner.models]
print(best_estimators)

[RandomForestClassifier(max_depth=3, min_samples_leaf=10, random_state=123), RandomForestClassifier(max_depth=3, min_samples_leaf=10, random_state=123)]


In [ ]:
best_estimators = [RandomForestClassifier(max_depth=3, min_samples_leaf=10, random_state=123), RandomForestClassifier(max_depth=3, min_samples_leaf=10, random_state=123)]

[RandomForestClassifier(max_depth=3, min_samples_leaf=10, random_state=123),
 RandomForestClassifier(max_depth=3, min_samples_leaf=10, random_state=123)]

In [ ]:
# Estimate treatment effects on test data
T_learner = TLearner(models=best_estimators)
T_learner.fit(Y1A_train, T_train, X=X_train, inference='bootstrap')
T_te = T_learner.effect(X_test)

In [ ]:
T_te_inference = T_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(T_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
       0.0         0.0   nan    nan           0.0           0.0
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
      0.0             0.0             0.0
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
         0.0            nan            nan
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 4.2 Calculate the robustness

In [ ]:
T_learner.fit(Y=Y1A_test, T=T_test, X=X_test)
T_te_test = T_learner.effect(X_test)

ERMSE = ((T_te-T_te_test)**2).mean()**(1/2)
print(ERMSE)

0.0


### 4.3 Calculate the R-Score 

In [ ]:
rscore = scorer_a.score(T_learner)
print(rscore)

-0.0010123335155265423


## 5. X-learner

### 5.1 Select best performing nuisance model class and hyperparameters

In [ ]:
# Instantiate the X_Learner
X_learner = XLearner(models=first_stage_clf(), propensity_model=first_stage_prop())

In [ ]:
# Train X_learner
X_learner.fit(Y1A_train, T_train, X=X_train)

In [ ]:
# Assuming that X_learner.models is a list of the fitted GridSearchCVList objects
best_estimators_models = [model.best_estimator_ for model in X_learner.models]

# best_estimators[0] contains the best LogisticRegression estimator
# best_estimators[1] contains the best RandomForestClassifier estimator
# best_estimators[2] contains the best GradientBoostingClassifier estimator
print(best_estimators_models)

[RandomForestClassifier(max_depth=3, min_samples_leaf=10, random_state=123), RandomForestClassifier(max_depth=3, min_samples_leaf=10, random_state=123)]


In [ ]:
best_estimators_models = [RandomForestClassifier(max_depth=3, min_samples_leaf=10, random_state=123), RandomForestClassifier(max_depth=3, min_samples_leaf=10, random_state=123)]

In [ ]:
best_estimator_pm = X_learner.propensity_models[0].best_estimator_
print(best_estimator_pm)

GradientBoostingClassifier(min_samples_leaf=30, random_state=123)


In [ ]:
best_estimator_pm = GradientBoostingClassifier(min_samples_leaf=30, random_state=123)

In [ ]:
# Estimate treatment effects on test data
X_learner = XLearner(models=best_estimators_models, propensity_model=best_estimator_pm)
X_learner.fit(Y1A_train, T_train, X=X_train, inference='bootstrap')
X_te = X_learner.effect(X_test)
X_te_inference = X_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(X_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
       0.0         0.0   nan    nan           0.0           0.0
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
      0.0             0.0             0.0
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
         0.0            nan            nan
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 5.2 Calculate the robustness

In [ ]:
X_learner.fit(Y=Y1A_test, T=T_test, X=X_test)
X_te_test = X_learner.effect(X_test)

ERMSE = ((X_te-X_te_test)**2).mean()**(1/2)
print(ERMSE)

0.0


### 5.3 Calculate the R-Score

In [ ]:
rscore = scorer_a.score(X_learner)
print(rscore)

-0.0010123335155265423


## 6. Causal Forest DML 

### 6.1 Select best performing nuisance model and hyperparameter

In [ ]:
forestdml = CausalForestDML(model_y=model_ya, model_t=model_ta, discrete_treatment=True,random_state=seed)

In [ ]:
forestdml.fit(Y1A_train, T_train, X=X_train)

In [ ]:
# Estimate treatment effects on test data
forestdml_te = forestdml.effect(X_test)
forestdml_te_inference = forestdml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(forestdml_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.003        0.09 0.039  0.969        -0.145         0.152
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.042          -0.055           0.078
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.099         -0.147          0.169
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 6.2 Calculate the robustness

In [ ]:
forestdml.fit(Y=Y1A_test, T=T_test, X=X_test)
forestdml_te_test = forestdml.effect(X_test)

ERMSE = ((forestdml_te-forestdml_te_test)**2).mean()**(1/2)
print(ERMSE)

0.07088373679216084


### 6.3 Calculate the RScore

In [ ]:
rscore = scorer_a.score(forestdml)
print(rscore)

0.030502537310160283


## 7. ForestDRLearner 

### 7.1 Select best performing nuisance model and hyperparameter

In [ ]:
forestdrl = ForestDRLearner(model_regression=model_yta, model_propensity=model_ta, random_state=seed)
forestdrl.fit(Y1A_train, T_train, X=X_train, cache_values=True)
# Estimate treatment effects on test data
forestdrl_te = forestdrl.effect(X_test)
forestdrl_te_inference = forestdrl.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(forestdrl_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.007       0.049 0.138  0.891        -0.073         0.087
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.023          -0.026           0.048
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.054         -0.072          0.099
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 7.2 Calculate the robustness

In [ ]:
forestdrl.fit(Y=Y1A_test, T=T_test, X=X_test)
forestdrl_te_test = forestdrl.effect(X_test)

ERMSE = ((forestdrl_te-forestdrl_te_test)**2).mean()**(1/2)
print(ERMSE)

0.054306500591324136


### 7.3 Calculate the RScore

In [ ]:
rscore = scorer_a.score(forestdrl)
print(rscore)

0.04274664160867936


## 8. NonParamDML 

### 8.1 Select best performing nuisance model and hyperparameter

In [ ]:
nonparamdml = NonParamDML(model_y=model_ya, model_t=model_ta, model_final=final_stage(),random_state=123)

In [ ]:
nonparamdml.fit(Y1A_train, T_train, X=X_train)

In [ ]:
# Access the best estimator for model_final after fitting the NonParamDML
best_estimator_model_final = nonparamdml.model_final_.best_estimator_
print(best_estimator_model_final)

RandomForestRegressor(max_depth=6, min_samples_leaf=50, random_state=123)


In [ ]:
best_estimator_model_final = RandomForestRegressor(max_depth=6, min_samples_leaf=50, random_state=123)

In [ ]:
nonparamdml = NonParamDML(model_y=model_yb, model_t=model_tb, model_final=best_estimator_model_final,random_state=123)
nonparamdml.fit(Y1B_train, T_train, X=X_train)

In [ ]:
nonparamdml_te = nonparamdml.effect(X_test)
nonparamdml_te_inference = nonparamdml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(nonparamdml_te_inference)

Uncertainty of Mean Point Estimate
mean_point
----------
    -0.019
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.043          -0.091            0.05
-----------------------------------------


### 8.2 Calculate the robustness

In [ ]:
nonparamdml.fit(Y=Y1A_test, T=T_test, X=X_test)
nonparamdml_te_test = nonparamdml.effect(X_test)

ERMSE = ((nonparamdml_te-nonparamdml_te_test)**2).mean()**(1/2)
print(ERMSE)

0.05687935008336561


### 8.3 Calculate the RScore

In [ ]:
rscore = scorer_a.score(nonparamdml)
print(rscore)

0.004358019819363457


# (B) Estimator selection: Continuous outcome


## 1. Linear DML 

### 1.1 Select the best performing nuisance model class and hyperparameters 

In [ ]:
ldml = LinearDML(model_y=model_yb,
                model_t=model_tb,
                random_state=123)

In [ ]:
ldml.fit(Y1B_train, T_train, X=X_train)

In [ ]:
ldml_cate = ldml.effect(X_test)
ldml_cate_inference = ldml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(ldml_cate_inference)

               Uncertainty of Mean Point Estimate               
mean_point stderr_mean zstat  pvalue ci_mean_lower ci_mean_upper
----------------------------------------------------------------
    -0.016       0.019 -0.848  0.397        -0.046         0.015
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.166          -0.253           0.216
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.167         -0.345          0.314
------------------------------------------


### 1.2 Calculate the robustness

In [ ]:
ldml.fit(Y=Y1B_test, T=T_test, X=X_test)
ldml_cate_test = ldml.effect(X_test)

ERMSE = ((ldml_cate-ldml_cate_test)**2).mean()**(1/2)
print(ERMSE)

0.37807011372246113


### 1.3 Calculate the RScore 

In [ ]:
rscore = scorer_b.score(ldml)
print(rscore)

0.029003069281162586


## 2. LinearDRLearner 

### 2.1 Select best performing nuisance model class and hyperparameters

In [ ]:
LinearDR_learner = LinearDRLearner(model_regression=model_ytb, model_propensity=model_tb, random_state=123)

In [ ]:
LinearDR_learner.fit(Y1B_train, T_train, X=X_train)

In [ ]:
LinearDR_learner_cate = LinearDR_learner.effect(X_test)

In [ ]:
LinearDR_learner_cate_inference = LinearDR_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(LinearDR_learner_cate_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.026        0.02 1.336  0.182        -0.006         0.058
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
     0.15          -0.208            0.27
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.151         -0.314          0.389
------------------------------------------


 ### 2.2 Calculate the robustness

In [ ]:
LinearDR_learner.fit(Y=Y1B_test, T=T_test, X=X_test)
LinearDR_learner_cate_test = LinearDR_learner.effect(X_test)

ERMSE = ((LinearDR_learner_cate-LinearDR_learner_cate_test)**2).mean()**(1/2)
print(ERMSE)

0.44592845584525975


### 2.3 Calculate the RScore


In [ ]:
rscore = scorer_b.score(LinearDR_learner)
print(rscore)

-0.02111385630035012


## 3. S-Learner 

### 3.1 Select best performing nuisance model class and hyperparameters

In [ ]:
# Instantiate the SLearner
S_learner = SLearner(overall_model= first_stage_reg())

In [ ]:
# Train S_learner
best_estimator_s_learner = S_learner.fit(Y1B_train, T_train, X=X_train).overall_model.best_estimator_
print(best_estimator_s_learner)

GradientBoostingRegressor(min_samples_leaf=50, random_state=123)


In [ ]:
best_estimator_s_learner = GradientBoostingRegressor(min_samples_leaf=50, random_state=123)

In [ ]:
# Estimate treatment effects on test data
S_learner = SLearner(overall_model=best_estimator_s_learner)
S_learner.fit(Y1B_train, T_train, X=X_train, inference='bootstrap')
S_te = S_learner.effect(X_test)

In [ ]:
S_te_inference = S_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(S_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
       0.0       0.012   0.0    1.0         -0.02          0.02
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
      0.0             0.0             0.0
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.012         -0.018          0.018
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 3.2 Calculate the robustness

In [ ]:
S_learner.fit(Y=Y1B_test, T=T_test, X=X_test)
S_te_test = S_learner.effect(X_test)

ERMSE = ((S_te-S_te_test)**2).mean()**(1/2)
print(ERMSE)

0.0


### 3.3 Calculate the RScore 

In [ ]:
rscore = scorer_b.score(S_learner)
print(rscore)

-0.0007788446049312725


## 4. T-Learner 

### 4.1 Select the best performing nuisance model class and hyperparameters

In [ ]:
# Instantiate the T_Learner
T_learner = TLearner(models=first_stage_reg())

In [ ]:
# Train T_learner
T_learner.fit(Y1B_train, T_train, X=X_train)

In [ ]:
# Assuming that T_learner.models is a list of the fitted GridSearchCVList objects
best_estimators = [model.best_estimator_ for model in T_learner.models]
print(best_estimators)

[GradientBoostingRegressor(min_samples_leaf=100, random_state=123), RandomForestRegressor(max_depth=6, min_samples_leaf=50, random_state=123)]


In [ ]:
best_estimators = [GradientBoostingRegressor(min_samples_leaf=100, random_state=123), RandomForestRegressor(max_depth=6, min_samples_leaf=50, random_state=123)]

In [ ]:
# Estimate treatment effects on test data
T_learner = TLearner(models=best_estimators)
T_learner.fit(Y1B_train, T_train, X=X_train, inference='bootstrap')
T_te = T_learner.effect(X_test)

In [ ]:
T_te_inference = T_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(T_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
      0.03       0.111 0.269  0.788        -0.152         0.212
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.126          -0.178           0.242
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.167         -0.238          0.312
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 4.2 Calculate the robustness 

In [ ]:
T_learner.fit(Y=Y1B_test, T=T_test, X=X_test)
T_te_test = T_learner.effect(X_test)

ERMSE = ((T_te-T_te_test)**2).mean()**(1/2)
print(ERMSE)

0.1872063610583569


### 4.3 Calculate the RScore 

In [ ]:
rscore = scorer_b.score(T_learner)
print(rscore)

0.13251803843958443


## 5. X-learner 


### 5.1 Select the best performing nuisance model class and hyperparameters 

In [ ]:
# Instantiate the X_Learner
X_learner = XLearner(models=first_stage_reg(), propensity_model=first_stage_prop())

In [ ]:
# Train X_learner
X_learner.fit(Y1B_train, T_train, X=X_train)

In [ ]:
# Assuming that X_learner.models is a list of the fitted GridSearchCVList objects
best_estimators_models = [model.best_estimator_ for model in X_learner.models]

# best_estimators[0] contains the best LogisticRegression estimator
# best_estimators[1] contains the best RandomForestClassifier estimator
# best_estimators[2] contains the best GradientBoostingClassifier estimator
print(best_estimators_models)

[GradientBoostingRegressor(min_samples_leaf=100, random_state=123), RandomForestRegressor(max_depth=6, min_samples_leaf=50, random_state=123)]


In [ ]:
best_estimators_models = [GradientBoostingRegressor(min_samples_leaf=100, random_state=123), RandomForestRegressor(max_depth=6, min_samples_leaf=50, random_state=123)]

In [ ]:
best_estimator_pm = X_learner.propensity_models[0].best_estimator_
print(best_estimator_pm)

GradientBoostingClassifier(min_samples_leaf=30, random_state=123)


In [ ]:
best_estimator_pm = GradientBoostingClassifier(min_samples_leaf=30, random_state=123)

In [ ]:
# Estimate treatment effects on test data
X_learner = XLearner(models=best_estimators_models, propensity_model=best_estimator_pm)
X_learner.fit(Y1B_train, T_train, X=X_train, inference='bootstrap')
X_te = X_learner.effect(X_test)
X_te_inference = X_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(X_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.025       0.065 0.378  0.706        -0.083         0.132
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.068          -0.069           0.145
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.094         -0.117          0.191
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 5.2 Calculate the robustness 

In [ ]:
X_learner.fit(Y=Y1B_test, T=T_test, X=X_test)
X_te_test = X_learner.effect(X_test)

ERMSE = ((X_te-X_te_test)**2).mean()**(1/2)
print(ERMSE)

0.10453046519115008


### 5.3 Calculate the RScore 

In [ ]:
rscore = scorer_b.score(X_learner)
print(rscore)

0.02177058613715277


## 6. NonParamDML 

### 6.1 Select the best performing nuisance model class and hyperparameters 

In [ ]:
nonparamdml = NonParamDML(model_y=model_yb, model_t=model_tb, model_final=final_stage(),random_state=123)

In [ ]:
nonparamdml.fit(Y1B_train, T_train, X=X_train)

In [ ]:
# Access the best estimator for model_final after fitting the NonParamDML
best_estimator_model_final = nonparamdml.model_final_.best_estimator_
print(best_estimator_model_final)

WeightedLasso(alpha=0.1, random_state=123)


In [ ]:
best_estimator_model_final = WeightedLasso(alpha=0.1, random_state=123)

In [ ]:
nonparamdml = NonParamDML(model_y=model_yb, model_t=model_tb, model_final=best_estimator_model_final,random_state=123)
nonparamdml.fit(Y1B_train, T_train, X=X_train)

In [ ]:
nonparamdml_te = nonparamdml.effect(X_test)
nonparamdml_te_inference = nonparamdml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(nonparamdml_te_inference)

Uncertainty of Mean Point Estimate
mean_point
----------
    -0.015
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
      0.0          -0.015          -0.015
-----------------------------------------


### 6.1 Calculate the robustness

In [ ]:
nonparamdml.fit(Y=Y1B_test, T=T_test, X=X_test)
nonparamdml_te_test = nonparamdml.effect(X_test)

ERMSE = ((nonparamdml_te-nonparamdml_te_test)**2).mean()**(1/2)
print(ERMSE)

0.0026244356141261506


### 6.3  Calculate the RScore

In [ ]:
rscore = scorer_b.score(nonparamdml)
print(rscore)

-0.00025747245085927517


## 7. CausalForestDML 

### 7.1 Select best performing nuisance model and hyperparameter

In [ ]:
forestdml = CausalForestDML(model_y=model_yb, model_t=model_tb, discrete_treatment=True, random_state=seed)

In [ ]:
forestdml.fit(Y1B_train, T_train, X=X_train)

In [ ]:
# Estimate treatment effects on test data
forestdml_te = forestdml.effect(X_test)
forestdml_te_inference = forestdml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(forestdml_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
      0.02       0.114 0.176  0.861        -0.168         0.208
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.048          -0.059             0.1
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.124         -0.178          0.223
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 6.2 Calculate the robustness

In [ ]:
forestdml.fit(Y=Y1B_test, T=T_test, X=X_test)
forestdml_te_test = forestdml.effect(X_test)

ERMSE = ((forestdml_te-forestdml_te_test)**2).mean()**(1/2)
print(ERMSE)

0.10081048596557782


### 6.3 Calculate the RScore

In [ ]:
rscore = scorer_b.score(forestdml)
print(rscore)

0.051221756437902055


## 8. ForestDRLearner 


### 7.1 Select best performing nuisance model and hyperparameter

In [ ]:
forestdrl = ForestDRLearner(model_regression=model_ytb, model_propensity=model_tb, random_state=seed)
forestdrl.fit(Y1B_train, T_train, X=X_train, cache_values=True)
# Estimate treatment effects on test data
forestdrl_te = forestdrl.effect(X_test)
forestdrl_te_inference = forestdrl.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(forestdrl_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.029       0.069 0.414  0.679        -0.085         0.142
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.026          -0.012           0.072
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.074         -0.087           0.15
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 7.2 Calculate the robustness

In [ ]:
forestdrl.fit(Y=Y1B_test, T=T_test, X=X_test)
forestdrl_te_test = forestdrl.effect(X_test)

ERMSE = ((forestdrl_te-forestdrl_te_test)**2).mean()**(1/2)
print(ERMSE)

0.1014686823449028


### 7.3 Calculate the RScore

In [ ]:
rscore = scorer_b.score(forestdrl)
print(rscore)

0.05808081567921053
